# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import colorsys

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [11]:
import utilities.helper
import trodes.read_exported

In [12]:
FONTSIZE = 20

In [13]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

In [14]:
EPHYS_SAMPLE_RATE = 20000

Explanation of each input and where it comes from.

In [15]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_3_alone_comp"

In [16]:
# TRIAL_LABELS_DF = pd.read_excel("/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_comp_ext/results/2024_06_26_sleap_clustering/data/rce_pilot_3_alone_comp_per_video_trial_labels.xlsx")
TRIAL_LABELS_DF = pd.read_excel("../../data/{}_per_video_trial_labels.xlsx".format(OUTPUT_PREFIX))

In [17]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.drop(columns=[col for col in TRIAL_LABELS_DF if "unnamed" in col.lower()], errors="ignore")

In [18]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.rename(columns={col: col.strip() for col in TRIAL_LABELS_DF}, errors="ignore")

In [19]:
LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_03_spectral_bands.pkl".format(OUTPUT_PREFIX))

In [20]:
# LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_04_spectral_and_sleap.pkl".format(OUTPUT_PREFIX))

In [21]:
LFP_SPECTRAL_DF = LFP_SPECTRAL_DF.drop(columns=[col for col in LFP_SPECTRAL_DF if "unnamed" in col.lower()], errors="ignore")

In [22]:
LFP_SPECTRAL_DF = LFP_SPECTRAL_DF.rename(columns={col: col.strip() for col in LFP_SPECTRAL_DF}, errors="ignore")

In [23]:
cluster_range_columns = pd.read_pickle("../2024_06_26_sleap_clustering/proc/cluster_range_columns.pkl")

In [24]:
FULL_LFP_TRACES_PKL = "{}_10_per_trial_spectral_bans_sleap.pkl".format(OUTPUT_PREFIX)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [25]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [26]:
LFP_SPECTRAL_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'notes', 'session_path',
       'recording', 'current_subject',
       ...
       'vHPC_power_theta', 'vHPC_power_gamma', 'BLA_power_theta',
       'BLA_power_gamma', 'LH_power_theta', 'LH_power_gamma', 'MD_power_theta',
       'MD_power_gamma', 'mPFC_power_theta', 'mPFC_power_gamma'],
      dtype='object', length=155)

In [27]:
TRIAL_LABELS_DF.columns

Index(['session_dir', 'tone_start_frame', 'reward_start', 'reward_dispensed',
       'tone_stop_frame', 'condition', 'competition_closeness',
       'get_reward_frame', 'out_reward_frame', 'notes', 'experiment',
       'box_1_port_entry_frames', 'box_2_port_entry_frames', 'video_name',
       'all_subjects', 'first_timestamp', 'last_timestamp',
       'tone_start_timestamp', 'tone_stop_timestamp',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps'],
      dtype='object')

## Merging the labels and LFP spectral

In [28]:
TRIAL_LABELS_DF.head()

,session_dir,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,notes,...,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,20240320_142408_alone_comp_subj_3-1_and_3-3,0.0,80,100.0,74.0,NaN,NaN,NaN,NaN,NaN,...,[ 0 74],[ 0 74],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,0.0,75474.0,[ 0 75474],[ 0 75474]
1,20240320_142408_alone_comp_subj_3-1_and_3-3,1272.0,1352,1372.0,1471.0,rewarded,NaN,NaN,NaN,NaN,...,[1282 1283] [1285 1297] [1326 1344] [1362 1509],[1326 1328] [1331 1347] [1380 1592],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,1275889.0,1475891.0,[1285888 1286688] [1287488 1300689] [1330286 1...,[1329489 1331487] [1334489 1350089] [1383490 1...
2,20240320_142408_alone_comp_subj_3-1_and_3-3,3268.0,3348,3368.0,3469.0,rewarded,NaN,NaN,NaN,NaN,...,[3281 3324] [3324 3327] [3344 3347] [3347 3523],[3327 3704],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,3275911.0,3475914.0,[3287914 3330314] [3331114 3333712] [3351715 3...,[3333515 3712519]
3,20240320_142408_alone_comp_subj_3-1_and_3-3,4466.0,4546,4566.0,4667.0,rewarded,NaN,NaN,NaN,NaN,...,[4441 4476] [4476 4489] [4508 4509] [4511 4534...,[4445 4473] [4494 4551] [4566 4830],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,4475927.0,4675931.0,[4450528 4485129] [4485929 4497529] [4516929 4...,[4454926 4483129] [4503531 4560730] [4575330 4...
4,20240320_142408_alone_comp_subj_3-1_and_3-3,5465.0,5545,5565.0,5664.0,rewarded,NaN,NaN,NaN,NaN,...,[5483 5512] [5552 5698],[5455 5486] [5581 5794],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,5475942.0,5675942.0,[5493742 5523542] [5563742 5709147],[5466941 5496741] [5592343 5806546]


In [29]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,notes,session_path,recording,current_subject,...,vHPC_power_theta,vHPC_power_gamma,BLA_power_theta,BLA_power_gamma,LH_power_theta,LH_power_gamma,MD_power_theta,MD_power_gamma,mPFC_power_theta,mPFC_power_gamma
0,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[0.01945638, nan, nan, nan, 0.019393958, nan, ...","[0.0130572505, nan, nan, nan, 0.007762581, nan...","[nan, nan, 0.03636455, 0.029700823, 0.02418904...","[nan, nan, 0.0018358257, 0.0017288909, 0.00135...","[0.033121496, 0.041370735, 0.055094175, 0.0427...","[0.0017061487, 0.0015902787, 0.0014763202, 0.0...","[0.016354451, 0.025327114, 0.028878175, 0.0222...","[0.00209533, 0.0015905038, 0.0014053435, 0.001..."
1,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[0.01945638, nan, nan, nan, 0.019393958, nan, ...","[0.0130572505, nan, nan, nan, 0.007762581, nan...","[nan, nan, 0.03636455, 0.029700823, 0.02418904...","[nan, nan, 0.0018358257, 0.0017288909, 0.00135...","[0.033121496, 0.041370735, 0.055094175, 0.0427...","[0.0017061487, 0.0015902787, 0.0014763202, 0.0...","[0.016354451, 0.025327114, 0.028878175, 0.0222...","[0.00209533, 0.0015905038, 0.0014053435, 0.001..."
2,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[0.0408452, 0.04246293, 0.04184469, nan, nan, ...","[0.0008264334, 0.0009773864, 0.0009832539, nan...","[0.024847934, 0.047361273, 0.042880192, nan, n...","[0.0006459263, 0.00090438855, 0.0010187915, na...","[0.023980794, 0.04565177, 0.03644787, nan, nan...","[0.00055783533, 0.00084574166, 0.0008706296, n...","[0.016893655, 0.027668845, 0.022960208, nan, n...","[0.0006401466, 0.0010481183, 0.0009893934, nan...","[0.021055607, 0.030588005, 0.023756983, nan, n...","[0.0008363458, 0.001157422, 0.0010688329, nan,..."
3,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[0.0408452, 0.04246293, 0.04184469, nan, nan, ...","[0.0008264334, 0.0009773864, 0.0009832539, nan...","[0.024847934, 0.047361273, 0.042880192, nan, n...","[0.0006459263, 0.00090438855, 0.0010187915, na...","[0.023980794, 0.04565177, 0.03644787, nan, nan...","[0.00055783533, 0.00084574166, 0.0008706296, n...","[0.016893655, 0.027668845, 0.022960208, nan, n...","[0.0006401466, 0.0010481183, 0.0009893934, nan...","[0.021055607, 0.030588005, 0.023756983, nan, n...","[0.0008363458, 0.001157422, 0.0010688329, nan,..."
4,rce_pilot_3,20240320_171038_alone_comp_subj_4-2_and_4-3,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79], [84

# Dropping redundant columns

In [30]:
dropping_columns = list(set(TRIAL_LABELS_DF.columns).intersection(set(LFP_SPECTRAL_DF.columns)))

In [31]:
dropping_columns

['session_dir',
 'video_name',
 'all_subjects',
 'last_timestamp',
 'box_1_port_entry_frames',
 'first_timestamp',
 'box_1_port_entry_timestamps',
 'notes',
 'box_2_port_entry_timestamps',
 'box_2_port_entry_frames']

In [32]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.drop(columns=[col for col in dropping_columns if col != "video_name"], errors="ignore")

## Merging the LFP and trial dataframe

In [33]:
LFP_SPECTRAL_DF["video_name"] = LFP_SPECTRAL_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))

In [34]:
LFP_SPECTRAL_DF["video_name"].unique()

array(['20240320_142408_alone_comp_subj_3-1_and_3-3.1',
       '20240320_142408_alone_comp_subj_3-1_and_3-3.2',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.1',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.2',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.1',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.2',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.1',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.2',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.1',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.2',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.1',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.2',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.1',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.2'], dtype=object)

In [35]:
LFP_SPECTRAL_DF["current_subject"] = LFP_SPECTRAL_DF["current_subject"].astype(str)


In [36]:
LFP_SPECTRAL_DF["current_subject"].head()

0    3.1
1    3.1
2    3.3
3    3.3
4    4.2
Name: current_subject, dtype: object

In [37]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["video_name"])

In [38]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["condition"])

In [39]:
TRIAL_LABELS_DF["video_name"] = TRIAL_LABELS_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))


In [40]:
TRIAL_LABELS_DF["video_name"].unique()

array(['20240320_142408_alone_comp_subj_3-1_and_3-3.1',
       '20240320_142408_alone_comp_subj_3-1_and_3-3.2',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.1',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.2',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.1',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.2',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.1',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.2',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.1',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.2',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.1',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.2',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.1',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.2'], dtype=object)

In [41]:
TRIALS_AND_SPECTRAL_DF = pd.merge(TRIAL_LABELS_DF, LFP_SPECTRAL_DF, on=["video_name"], how="inner", suffixes=('', '_remove'))

In [42]:
TRIALS_AND_SPECTRAL_DF[[col for col in TRIALS_AND_SPECTRAL_DF if "_remove" in col]].head()

""
0
1
2
3
4


In [43]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF if "remove" in col], errors="ignore")

In [44]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF if "Unnamed" in col])

In [45]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,vHPC_power_theta,vHPC_power_gamma,BLA_power_theta,BLA_power_gamma,LH_power_theta,LH_power_gamma,MD_power_theta,MD_power_gamma,mPFC_power_theta,mPFC_power_gamma
0,1272.0,1352,1372.0,1471.0,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[0.01945638, nan, nan, nan, 0.019393958, nan, ...","[0.0130572505, nan, nan, nan, 0.007762581, nan...","[nan, nan, 0.03636455, 0.029700823, 0.02418904...","[nan, nan, 0.0018358257, 0.0017288909, 0.00135...","[0.033121496, 0.041370735, 0.055094175, 0.0427...","[0.0017061487, 0.0015902787, 0.0014763202, 0.0...","[0.016354451, 0.025327114, 0.028878175, 0.0222...","[0.00209533, 0.0015905038, 0.0014053435, 0.001..."
1,1272.0,1352,1372.0,1471.0,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[0.0408452, 0.04246293, 0.04184469, nan, nan, ...","[0.0008264334, 0.0009773864, 0.0009832539, nan...","[0.024847934, 0.047361273, 0.042880192, nan, n...","[0.0006459263, 0.00090438855, 0.0010187915, na...","[0.023980794, 0.04565177, 0.03644787, nan, nan...","[0.00055783533, 0.00084574166, 0.0008706296, n...","[0.016893655, 0.027668845, 0.022960208, nan, n...","[0.0006401466, 0.0010481183, 0.0009893934, nan...","[0.021055607, 0.030588005, 0.023756983, nan, n...","[0.0008363458, 0.001157422, 0.0010688329, nan,..."
2,3268.0,3348,3368.0,3469.0,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[0.01945638, nan, nan, nan, 0.019393958, nan, ...","[0.0130572505, nan, nan, nan, 0.007762581, nan...","[nan, nan, 0.03636455, 0.029700823, 0.02418904...","[nan, nan, 0.0018358257, 0.0017288909, 0.00135...","[0.033121496, 0.041370735, 0.055094175, 0.0427...","[0.0017061487, 0.0015902787, 0.0014763202, 0.0...","[0.016354451, 0.025327114, 0.028878175, 0.0222...","[0.00209533, 0.0015905038, 0.0014053435, 0.001..."
3,3268.0,3348,3368.0,3469.0,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[0.0408452, 0.04246293, 0.04184469, nan, nan, ...","[0.0008264334, 0.0009773864, 0.0009832539, nan...","[0.024847934, 0.047361273, 0.042880192, nan, n...","[0.0006459263, 0.00090438855, 0.0010187915, na...","[0.023980794, 0.04565177, 0.03644787, nan, nan...","[0.00055783533, 0.00084574166, 0.0008706296, n...","[0.016893655, 0.027668845, 0.022960208, nan, n...","[0.0006401466, 0.0010481183, 0.0009893934, nan...","[0.021055607, 0.030588005, 0.023756983, nan, n...","[0.0008363458, 0.001157422, 0.0010688329, nan,..."
4,4466.0,4546,4566.0,4667.0,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[0.01945638, nan, nan, nan, 0.019393958, nan, ...","[0.0130572505, nan, nan, nan, 0.007762581, nan...","[nan, nan, 0.03636455, 0.029700823, 0.02418904...","[nan, nan, 0.0018358257, 0.0017288909, 0.00135...","[0.033121496, 0.041370735, 0.055094175, 0.0427...","[0.0017061487, 0.0015902787, 0.0014763202, 0.0...","[0.016354451, 0.025327114, 0.028878175, 0.0222...","[0.00209533, 0.0015905038, 0.0014053435, 0.001..."


In [46]:
TRIALS_AND_SPECTRAL_DF.tail()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,vHPC_power_theta,vHPC_power_gamma,BLA_power_theta,BLA_power_gamma,LH_power_theta,LH_power_gamma,MD_power_theta,MD_power_gamma,mPFC_power_theta,mPFC_power_gamma
871,25417.0,25497,25517.0,25616.0,rewarded,NaN,NaN,NaN,alone_comp,20240323_165815_alone_comp_subj_4-2_and_4-4.2,...,"[0.007144761, 0.0037135014, 0.005482981, 0.006...","[0.00016188143, 0.0002133701, 0.00021314951, 0...","[0.006264279, 0.0038419755, 0.0057587656, 0.00...","[0.00033974194, 0.00023258351, 0.0003284482, 0...","[0.013417456, 0.008575149, 0.015885003, 0.0151...","[0.00036998998, 0.00047189702, 0.00043293624, ...","[nan, 0.013873596, 0.0078123533, 0.009779366, ...","[nan, 0.0020573323, 0.0020168012, 0.0014864316...","[0.039393336, 0.030886643, 0.031441703, 0.0191...","[0.0024420302, 0.0033522763, 0.0032580034, 0.0..."
872,26515.0,26595,26615.0,26715.0,rewarded,NaN,NaN,NaN,alone_comp,20240323_165815_alone_comp_subj_4-2_and_4-4.2,...,"[0.014044589, 0.025243934, 0.021230606, 0.0120...","[0.003164349, 0.0026817215, 0.002723993, 0.002...","[0.046375997, 0.03196241, 0.033612445, 0.02650...","[0.0023618669, 0.0015799376, 0.0016996932, 0.0...","[0.06547418, nan, nan, nan, nan, nan, 0.060413...","[0.0015344078, nan, nan, nan, nan, nan, 0.0014...","[0.051771734, nan, nan, 0.0470939, 0.08267944,...","[0.0010779751, nan, nan, 0.00082194566, 0.0014...","[0.043606173, 0.032464292, 0.014611534, 0.0237...","[0.0014644166, 0.0020225544, 0.0025327585, 0.0..."
873,26515.0,26595,26615.0,26715.0,rewarded,NaN,NaN,NaN,alone_comp,20240323_165815_alone_comp_subj_4-2_and_4-4.2,...,"[0.007144761, 0.0037135014, 0.005482981, 0.006...","[0.00016188143, 0.0002133701, 0.00021314951, 0...","[0.006264279, 0.0038419755, 0.0057587656, 0.00...","[0.00033974194, 0.00023258351, 0.0003284482, 0...","[0.013417456, 0.008575149, 0.015885003, 0.0151...","[0.00036998998, 0.00047189702, 0.00043293624, ...","[nan, 0.013873596, 0.0078123533, 0.009779366, ...","[nan, 0.0020573323, 0.0020168012, 0.0014864316...","[0.039393336, 0.030886643, 0.031441703, 0.0191...","[0.0024420302, 0.0033522763, 0.0032580034, 0.0..."
874,28211.0,28291,28311.0,28412.0,rewarded,NaN,NaN,NaN,alone_comp,20240323_165815_alone_comp_subj_4-2_and_4-4.2,...,"[0.014044589, 0.025243934, 0.021230606, 0.0120...","[0.003164349, 0.0026817215, 0.002723993, 0.002...","[0.046375997, 0.03196241, 0.033612445, 0.02650...","[0.0023618669, 0.0015799376, 0.0016996932, 0.0...","[0.06547418, nan, nan, nan, nan, nan, 0.060413...","[0.0015344078, nan, nan, nan, nan, nan, 0.0014...","[0.051771734, nan, nan, 0.0470939, 0.08267944,...","[0.0010779751, nan, nan, 0.00082194566, 0.0014...","[0.043606173, 0.032464292, 0.014611534, 0.0237...","[0.0014644166, 0.0020225544, 0.0025327585, 0.0..."
875,28211.0,28291,28311.0,28412.0,rewarded,NaN,NaN,NaN,alone_comp,20240323_165815_alone_comp_subj_4-2_and_4-4.2,...,"[0.007144761, 0.0037135014, 0.005482981, 0.006...","[0.00016188143, 0.0002133701, 0.00021314951, 0...","[0.006264279, 0.0038419755, 0.0057587656, 0.00...","[0.00033974194, 0.00023258351, 0.0003284482, 0...","[0.013417456, 0.008575149, 0.015885003, 0.0151...","[0.00036998998, 0.00047189702, 0.00043293624, ...","[nan, 0.013873596, 0.0078123533, 0.009779366, ...","[nan, 0.0020573323, 0.0020168012, 0.0014864316...","[0.039393336, 0.030886643, 0.031441703, 0.0191...","[0.0024420302, 0.0033522763, 0.0032580034, 0.0..."


In [47]:
timestamps_col = [col for col in TRIALS_AND_SPECTRAL_DF if "timestamp" in col and "timestamps" not in col and "cluster" not in col]

In [48]:
for col in timestamps_col:
    TRIALS_AND_SPECTRAL_DF[col] = TRIALS_AND_SPECTRAL_DF[col].astype(int)

In [49]:
TRIALS_AND_SPECTRAL_DF[timestamps_col].head()

,tone_start_timestamp,tone_stop_timestamp,first_timestamp,last_timestamp
0,1275889,1475891,1830734,65425515
1,1275889,1475891,1830734,65425515
2,3275911,3475914,1830734,65425515
3,3275911,3475914,1830734,65425515
4,4475927,4675931,1830734,65425515


In [50]:
frame_col = [col for col in TRIALS_AND_SPECTRAL_DF if "frame" in col and "tone" in col and "frames" not in col]

In [51]:
for col in frame_col:
    TRIALS_AND_SPECTRAL_DF[col] = TRIALS_AND_SPECTRAL_DF[col].astype(int)

In [52]:
TRIALS_AND_SPECTRAL_DF[frame_col].head()

,tone_start_frame,tone_stop_frame
0,1272,1471
1,1272,1471
2,3268,3469
3,3268,3469
4,4466,4667


## Adding the SLEAP data

In [53]:
cluster_range_columns.columns

Index(['video_name', 'current_subject', 'tone_start_frame', 'agent_locations',
       'agent_nose', 'agent_nose_to_reward_port', 'agent_tail_base',
       'agent_thorax', 'agent_thorax_velocity', 'agent_to_reward_port_angle',
       'closebool_agent_nose_to_reward_port',
       'closebool_subject_nose_to_reward_port', 'clusterable_embedding_x',
       'clusterable_embedding_y', 'competitiveness', 'frame_index',
       'kmeans_cluster', 'manual_cluster_id',
       'movingbool_agent_thorax_velocity',
       'movingbool_subject_thorax_velocity', 'nose_to_reward_port_diff',
       'nose_to_reward_port_sum', 'nose_to_tail_diff', 'nose_to_tail_sum',
       'standard_embedding_x', 'standard_embedding_y', 'subject_locations',
       'subject_nose', 'subject_nose_to_reward_port', 'subject_tail_base',
       'subject_thorax', 'subject_thorax_to_agent_thorax',
       'subject_thorax_velocity', 'subject_to_reward_port_angle',
       'thorax_velocity_diff', 'thorax_velocity_sum',
       'to_reward_

In [54]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,vHPC_power_theta,vHPC_power_gamma,BLA_power_theta,BLA_power_gamma,LH_power_theta,LH_power_gamma,MD_power_theta,MD_power_gamma,mPFC_power_theta,mPFC_power_gamma
0,1272,1352,1372.0,1471,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[0.01945638, nan, nan, nan, 0.019393958, nan, ...","[0.0130572505, nan, nan, nan, 0.007762581, nan...","[nan, nan, 0.03636455, 0.029700823, 0.02418904...","[nan, nan, 0.0018358257, 0.0017288909, 0.00135...","[0.033121496, 0.041370735, 0.055094175, 0.0427...","[0.0017061487, 0.0015902787, 0.0014763202, 0.0...","[0.016354451, 0.025327114, 0.028878175, 0.0222...","[0.00209533, 0.0015905038, 0.0014053435, 0.001..."
1,1272,1352,1372.0,1471,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[0.0408452, 0.04246293, 0.04184469, nan, nan, ...","[0.0008264334, 0.0009773864, 0.0009832539, nan...","[0.024847934, 0.047361273, 0.042880192, nan, n...","[0.0006459263, 0.00090438855, 0.0010187915, na...","[0.023980794, 0.04565177, 0.03644787, nan, nan...","[0.00055783533, 0.00084574166, 0.0008706296, n...","[0.016893655, 0.027668845, 0.022960208, nan, n...","[0.0006401466, 0.0010481183, 0.0009893934, nan...","[0.021055607, 0.030588005, 0.023756983, nan, n...","[0.0008363458, 0.001157422, 0.0010688329, nan,..."
2,3268,3348,3368.0,3469,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[0.01945638, nan, nan, nan, 0.019393958, nan, ...","[0.0130572505, nan, nan, nan, 0.007762581, nan...","[nan, nan, 0.03636455, 0.029700823, 0.02418904...","[nan, nan, 0.0018358257, 0.0017288909, 0.00135...","[0.033121496, 0.041370735, 0.055094175, 0.0427...","[0.0017061487, 0.0015902787, 0.0014763202, 0.0...","[0.016354451, 0.025327114, 0.028878175, 0.0222...","[0.00209533, 0.0015905038, 0.0014053435, 0.001..."
3,3268,3348,3368.0,3469,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[0.0408452, 0.04246293, 0.04184469, nan, nan, ...","[0.0008264334, 0.0009773864, 0.0009832539, nan...","[0.024847934, 0.047361273, 0.042880192, nan, n...","[0.0006459263, 0.00090438855, 0.0010187915, na...","[0.023980794, 0.04565177, 0.03644787, nan, nan...","[0.00055783533, 0.00084574166, 0.0008706296, n...","[0.016893655, 0.027668845, 0.022960208, nan, n...","[0.0006401466, 0.0010481183, 0.0009893934, nan...","[0.021055607, 0.030588005, 0.023756983, nan, n...","[0.0008363458, 0.001157422, 0.0010688329, nan,..."
4,4466,4546,4566.0,4667,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[0.01945638, nan, nan, nan, 0.019393958, nan, ...","[0.0130572505, nan, nan, nan, 0.007762581, nan...","[nan, nan, 0.03636455, 0.029700823, 0.02418904...","[nan, nan, 0.0018358257, 0.0017288909, 0.00135...","[0.033121496, 0.041370735, 0.055094175, 0.0427...","[0.0017061487, 0.0015902787, 0.0014763202, 0.0...","[0.016354451, 0.025327114, 0.028878175, 0.0222...","[0.00209533, 0.0015905038, 0.0014053435, 0.001..."


In [55]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    1275889
1    1275889
2    3275911
3    3275911
4    4475927
Name: tone_start_timestamp, dtype: int64

In [56]:
cluster_range_columns["tone_start_timestamp"] = cluster_range_columns["tone_start_timestamp"].astype(int)
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].astype(int)

In [57]:
cluster_range_columns["current_subject"] = cluster_range_columns["current_subject"].apply(lambda x: str(x).strip())
TRIALS_AND_SPECTRAL_DF["current_subject"] = TRIALS_AND_SPECTRAL_DF["current_subject"].apply(lambda x: str(x).strip())

In [58]:
TRIALS_AND_SPECTRAL_DF = pd.merge(TRIALS_AND_SPECTRAL_DF, cluster_range_columns, on=["video_name", "current_subject", "tone_start_timestamp"], how="left", suffixes=('', '_remove'))

In [59]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"]

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,stop_frame,tone_start_to_stop_frame,tone_stop_frame_remove,tone_stop_timestamp_remove,tracked_subject,video_id,cluster_index_ranges_dict,competitiveness_index_ranges_dict,cluster_frame_ranges_dict,competitiveness_frame_ranges_dict
42,34240,34320,34340.0,34440,3.1,Subj 1 Only,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,63504.0,"(34240, 34440)",34440.0,34476299.0,"[3.1, 3.3]",7.0,"{'0': [(0, 4), (201, 217), (224, 236), (250, 2...","{'competitive': [(0, 5), (201, 217), (224, 258...","{'0': [(34040, 34044), (34241, 34257), (34264,...","{'competitive': [(34040, 34045), (34241, 34257..."
43,34240,34320,34340.0,34440,3.1,Subj 1 Only,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,63504.0,"(34240, 34440)",34440.0,34476299.0,"[3.1, 3.3]",7.0,"{'0': [(0, 4), (201, 217), (224, 236), (250, 2...","{'competitive': [(0, 5), (201, 217), (224, 258...","{'0': [(34040, 34044), (34241, 34257), (34264,...","{'competitive': [(34040, 34045), (34241, 34257..."
44,35238,35318,35338.0,35437,3.1,Subj 1 then Subj 2,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,63504.0,"(35238, 35437)",35437.0,35476311.0,"[3.1, 3.3]",7.0,"{'2': [(0, 10), (12, 16), (129, 134)], '4': [(...","{'no_comp': [(0, 52), (129, 134), (163, 210), ...","{'2': [(35038, 35048), (35050, 35054), (35167,...","{'no_comp': [(35038, 35090), (35167, 35172), (..."
45,35238,35318,35338.0,35437,3.1,Subj 1 then Subj 2,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,63504.0,"(35238, 35437)",35437.0,35476311.0,"[3.1, 3.3]",7.0,"{'2': [(0, 10), (12, 16), (129, 134)], '4': [(...","{'no_comp': [(0, 52), (129, 134), (163, 210), ...","{'2': [(35038, 35048), (35050, 35054), (35167,...","{'no_comp': [(35038, 35090), (35167, 35172), (..."
46,36337,36417,36437.0,36536,3.3,Subj 2 Only,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,63504.0,"(36337, 36536)",36536.0,36576327.0,"[3.1, 3.3]",7.0,"{'6': [(0, 50), (59, 117), (128, 230), (372, 3...","{'no_comp': [(0, 249), (301, 366), (372, 379),...","{'6': [(36137, 36187), (36196, 36254), (36265,...","{'no_comp': [(36137, 36386), (36438, 36503), (..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,57880,57960,57980.0,58079,4.4,Subj 2 then Subj 1,NaN,NaN,alone_comp,20240323_165815_alone_comp_subj_4-2_and_4-4.1,...,63903.0,"(57880, 58079)",58079.0,58164893.0,"[4.2, 4.4]",13.0,"{'4': [(0, 67)], '1': [(68, 90), (322, 331)], ...","{'no_comp': [(0, 90), (286, 331)], 'competitiv...","{'4': [(57680, 57747)], '1': [(57748, 57770), ...","{'no_comp': [(57680, 57770), (57966, 58011)], ..."
830,60077,60157,60177.0,60276,4.4,Subj 2 blocking Subj 1,NaN,NaN,alone_comp,20240323_165815_alone_comp_subj_4-2_and_4-4.1,...,63903.0,"(60077, 60276)",60276.0,60364919.0,"[4.2, 4.4]",13.0,"{'2': [(0, 7), (21, 63), (156, 204), (247, 272...","{'no_comp': [(0, 7), (21, 74), (156, 204), (24...","{'2': [(59877, 59884), (59898, 59940), (60033,...","{'no_comp': [(59877, 59884), (59898, 59951), (..."
831,60077,60157,60177.0,60276,4.4,Subj 2 blocking Subj 1,NaN,NaN,alone_comp,20240323_165815_alone_comp_subj_4-2_and_4-4.1,...,63903.0,"(60077, 60276)",60276.0,60364919.0,"[4.2, 4.4]",13.0,"{'2': [(0, 7), (21, 63), (156, 204), (247, 272...","{'no_comp': [(0, 7), (21, 74), (156, 204), (24...","{'2': [(59877, 59884), (59898, 59940), (60033,...","{'no_comp': [(59877, 59884), (59898, 59951), (..."
832,61174,61254,61274.0,61375,tie,Close Call,NaN,NaN,alone_comp,20240323_165815_alone_comp_subj_4-2_and_4-4.1,...,63903.0,"(61174, 61375)",61375.0,61464932.0,"[4.2, 4.4]",13.0,"{'2': [(0, 34), (51, 72), (85, 88), (223, 231)...","{'no_comp': [(0, 133), (220, 234)], 'competiti...","{'2': [(60974, 61008), (61025, 61046), (61059,...","{'no_comp': [(60974, 61107), (61194, 61208)], ..."


# Calculating the relevant timestamps

In [60]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] - 10 * EPHYS_SAMPLE_RATE

In [61]:
TRIALS_AND_SPECTRAL_DF["post_trial_end_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"] + 10 * EPHYS_SAMPLE_RATE

In [62]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    1275889
1    1275889
2    3275911
3    3275911
4    4475927
Name: tone_start_timestamp, dtype: int64

In [63]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    1275889
1    1275889
2    3275911
3    3275911
4    4475927
Name: tone_start_timestamp, dtype: int64

In [64]:
TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"].head()

0    1475891
1    1475891
2    3475914
3    3475914
4    4675931
Name: tone_stop_timestamp, dtype: int64

In [65]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,tone_stop_frame_remove,tone_stop_timestamp_remove,tracked_subject,video_id,cluster_index_ranges_dict,competitiveness_index_ranges_dict,cluster_frame_ranges_dict,competitiveness_frame_ranges_dict,baseline_start_timestamp,post_trial_end_timestamp
0,1272,1352,1372.0,1471,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1075889,1675891
1,1272,1352,1372.0,1471,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1075889,1675891
2,3268,3348,3368.0,3469,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3075911,3675914
3,3268,3348,3368.0,3469,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3075911,3675914
4,4466,4546,4566.0,4667,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4275927,4875931


# Filtering out the SLEAP posed for during trials

In [66]:
TRIALS_AND_SPECTRAL_DF["video_frame"] = TRIALS_AND_SPECTRAL_DF["video_timestamps"].apply(lambda x: np.array(list(range(len(x)))) + 1)

In [67]:
TRIALS_AND_SPECTRAL_DF["video_frame"].head().apply(lambda x: x.shape)

0    (63504,)
1    (63504,)
2    (63504,)
3    (63504,)
4    (63504,)
Name: video_frame, dtype: object

In [68]:
cluster_range_columns.columns

Index(['video_name', 'current_subject', 'tone_start_frame', 'agent_locations',
       'agent_nose', 'agent_nose_to_reward_port', 'agent_tail_base',
       'agent_thorax', 'agent_thorax_velocity', 'agent_to_reward_port_angle',
       'closebool_agent_nose_to_reward_port',
       'closebool_subject_nose_to_reward_port', 'clusterable_embedding_x',
       'clusterable_embedding_y', 'competitiveness', 'frame_index',
       'kmeans_cluster', 'manual_cluster_id',
       'movingbool_agent_thorax_velocity',
       'movingbool_subject_thorax_velocity', 'nose_to_reward_port_diff',
       'nose_to_reward_port_sum', 'nose_to_tail_diff', 'nose_to_tail_sum',
       'standard_embedding_x', 'standard_embedding_y', 'subject_locations',
       'subject_nose', 'subject_nose_to_reward_port', 'subject_tail_base',
       'subject_thorax', 'subject_thorax_to_agent_thorax',
       'subject_thorax_velocity', 'subject_to_reward_port_angle',
       'thorax_velocity_diff', 'thorax_velocity_sum',
       'to_reward_

In [69]:
sleap_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if ("thorax" in col or "nose" in col or "standard_embedding" in col or "cluster" in col or "frame_index" in col or "locations" in col or "tailbase" in col or "competitiveness" in col) and "timestamp" not in col and "dict" not in col]


In [70]:
sleap_columns

['agent_locations',
 'agent_nose',
 'agent_nose_to_reward_port',
 'agent_thorax',
 'agent_thorax_velocity',
 'closebool_agent_nose_to_reward_port',
 'closebool_subject_nose_to_reward_port',
 'clusterable_embedding_x',
 'clusterable_embedding_y',
 'competitiveness',
 'frame_index',
 'kmeans_cluster',
 'manual_cluster_id',
 'movingbool_agent_thorax_velocity',
 'movingbool_subject_thorax_velocity',
 'nose_to_reward_port_diff',
 'nose_to_reward_port_sum',
 'nose_to_tail_diff',
 'nose_to_tail_sum',
 'standard_embedding_x',
 'standard_embedding_y',
 'subject_locations',
 'subject_nose',
 'subject_nose_to_reward_port',
 'subject_thorax',
 'subject_thorax_to_agent_thorax',
 'subject_thorax_velocity',
 'thorax_velocity_diff',
 'thorax_velocity_sum',
 'within_trial_frame_index']

In [71]:
for col in sleap_columns:
    print(col)
    print(TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"][col].apply(lambda x: x.shape if isinstance(x, np.ndarray) else np.nan).head())

agent_locations
42    (600, 6, 2)
43    (600, 6, 2)
44    (599, 6, 2)
45    (599, 6, 2)
46    (599, 6, 2)
Name: agent_locations, dtype: object
agent_nose
42    (600, 2)
43    (600, 2)
44    (599, 2)
45    (599, 2)
46    (599, 2)
Name: agent_nose, dtype: object
agent_nose_to_reward_port
42    (600,)
43    (600,)
44    (599,)
45    (599,)
46    (599,)
Name: agent_nose_to_reward_port, dtype: object
agent_thorax
42    (600, 2)
43    (600, 2)
44    (599, 2)
45    (599, 2)
46    (599, 2)
Name: agent_thorax, dtype: object
agent_thorax_velocity
42    (600,)
43    (600,)
44    (599,)
45    (599,)
46    (599,)
Name: agent_thorax_velocity, dtype: object
closebool_agent_nose_to_reward_port
42    (600,)
43    (600,)
44    (599,)
45    (599,)
46    (599,)
Name: closebool_agent_nose_to_reward_port, dtype: object
closebool_subject_nose_to_reward_port
42    (600,)
43    (600,)
44    (599,)
45    (599,)
46    (599,)
Name: closebool_subject_nose_to_reward_port, dtype: object
clusterable_embedding_x
42   

In [72]:
TRIALS_AND_SPECTRAL_DF["frame_index"]

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
871    NaN
872    NaN
873    NaN
874    NaN
875    NaN
Name: frame_index, Length: 876, dtype: object

In [73]:
TRIALS_AND_SPECTRAL_DF["cluster_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: x["video_timestamps"][x["frame_index"]] if isinstance(x["frame_index"], np.ndarray) else np.nan, axis=1)

In [74]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"]["cluster_timestamp"].iloc[0]

array([34077145, 34077145, 34078531, 34079917, 34079917, 34081303,
       34082688, 34084074, 34084074, 34085460, 34086846, 34086846,
       34088232, 34089618, 34091004, 34091004, 34092390, 34093775,
       34095161, 34095161, 34096547, 34097933, 34099319, 34099319,
       34100705, 34102091, 34102091, 34103476, 34104862, 34106248,
       34106248, 34107634, 34109020, 34109020, 34110406, 34111792,
       34113178, 34113178, 34114563, 34115949, 34115949, 34117335,
       34118721, 34120107, 34120107, 34121493, 34122879, 34124265,
       34124265, 34125650, 34127036, 34128422, 34128422, 34129808,
       34131194, 34131194, 34132580, 34133966, 34133966, 34135351,
       34136737, 34138123, 34138123, 34139509, 34140895, 34140895,
       34142281, 34143667, 34145053, 34145053, 34146438, 34147824,
       34149210, 34149210, 34150596, 34151982, 34153368, 34153368,
       34154754, 34156140, 34156140, 34157525, 34158911, 34160297,
       34160297, 34161683, 34163069, 34163069, 34164455, 34165

In [75]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"]["cluster_timestamp"].iloc[0].shape

(600,)

In [76]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"]

0       1275889
1       1275889
2       3275911
3       3275911
4       4475927
         ...   
871    25464497
872    26564508
873    26564508
874    28264531
875    28264531
Name: tone_start_timestamp, Length: 876, dtype: int64

In [77]:
sleap_columns

['agent_locations',
 'agent_nose',
 'agent_nose_to_reward_port',
 'agent_thorax',
 'agent_thorax_velocity',
 'closebool_agent_nose_to_reward_port',
 'closebool_subject_nose_to_reward_port',
 'clusterable_embedding_x',
 'clusterable_embedding_y',
 'competitiveness',
 'frame_index',
 'kmeans_cluster',
 'manual_cluster_id',
 'movingbool_agent_thorax_velocity',
 'movingbool_subject_thorax_velocity',
 'nose_to_reward_port_diff',
 'nose_to_reward_port_sum',
 'nose_to_tail_diff',
 'nose_to_tail_sum',
 'standard_embedding_x',
 'standard_embedding_y',
 'subject_locations',
 'subject_nose',
 'subject_nose_to_reward_port',
 'subject_thorax',
 'subject_thorax_to_agent_thorax',
 'subject_thorax_velocity',
 'thorax_velocity_diff',
 'thorax_velocity_sum',
 'within_trial_frame_index']

In [78]:
for col in sorted(sleap_columns):
    updated_item_col = "trial_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_video_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["cluster_timestamp"], items=x[col])[1] if isinstance(x["cluster_timestamp"], np.ndarray) else np.nan, axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["cluster_timestamp"], items=x[col])[0] if isinstance(x["cluster_timestamp"], np.ndarray) else np.nan, axis=1)

trial_agent_locations
trial_agent_nose
trial_agent_nose_to_reward_port
trial_agent_thorax
trial_agent_thorax_velocity
trial_closebool_agent_nose_to_reward_port
trial_closebool_subject_nose_to_reward_port
trial_clusterable_embedding_x
trial_clusterable_embedding_y
trial_competitiveness
trial_frame_index
trial_kmeans_cluster
trial_manual_cluster_id
trial_movingbool_agent_thorax_velocity
trial_movingbool_subject_thorax_velocity
trial_nose_to_reward_port_diff
trial_nose_to_reward_port_sum
trial_nose_to_tail_diff
trial_nose_to_tail_sum
trial_standard_embedding_x
trial_standard_embedding_y
trial_subject_locations
trial_subject_nose
trial_subject_nose_to_reward_port
trial_subject_thorax
trial_subject_thorax_to_agent_thorax
trial_subject_thorax_velocity
trial_thorax_velocity_diff
trial_thorax_velocity_sum
trial_within_trial_frame_index


In [79]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"].head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,trial_subject_locations,trial_subject_nose,trial_subject_nose_to_reward_port,trial_subject_thorax,trial_subject_thorax_to_agent_thorax,trial_subject_thorax_velocity,trial_thorax_velocity_diff,trial_thorax_velocity_sum,trial_within_trial_frame_index,trial_video_timestamps
42,34240,34320,34340.0,34440,3.1,Subj 1 Only,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[[[62.57065705282688, 33.44730809206244], [58....","[[60.54818493730512, 35.74634421564697], [60.8...","[0.28861773, 0.17335962, 0.17028531, 0.2535302...","[[59.8319529727617, 31.001132715032707], [59.8...","[12.89103, 12.176237, 11.61231, 11.06019, 10.5...","[-0.021209748, 0.003994167, 0.15215835, 0.4798...","[26.120943, 23.606047, 20.731367, 17.724455, 1...","[27.127563, 25.086893, 22.796099, 20.469105, 1...","[200, 201, 202, 203, 204, 205, 206, 207, 208, ...","[34276710, 34278096, 34279482, 34279482, 34280..."
43,34240,34320,34340.0,34440,3.1,Subj 1 Only,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[[[70.76603142951862, 27.905445222198825], [66...","[[67.33614650729025, 29.551449212479824], [66....","[9.162211, 8.083501, 7.0652747, 6.1474023, 5.3...","[[69.77804561944015, 24.819920337934217], [70....","[12.89103, 12.176237, 11.61231, 11.06019, 10.5...","[27.148773, 25.0829, 22.643942, 19.98921, 17.2...","[26.120943, 23.606047, 20.731367, 17.724455, 1...","[27.127563, 25.086893, 22.796099, 20.469105, 1...","[200, 201, 202, 203, 204, 205, 206, 207, 208, ...","[34276710, 34278096, 34279482, 34279482, 34280..."
44,35238,35318,35338.0,35437,3.1,Subj 1 then Subj 2,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[[[62.27856458758486, 33.78491219048569], [58....","[[60.821855153064945, 35.12897321756974], [60....","[0.41515765, 0.42165327, 0.43931708, 0.4439942...","[[63.76963180256321, 31.344854927989847], [63....","[12.874714, 12.780252, 12.830084, 12.994184, 1...","[0.5843707, 0.5663497, 0.55942494, 0.56991917,...","[26.742891, 27.9868, 29.002823, 29.783545, 30....","[27.911633, 29.1195, 30.121674, 30.923384, 31....","[200, 201, 202, 203, 204, 205, 206, 207, 208, ...","[35277308, 35277308, 35278694, 35280080, 35280..."
45,35238,35318,35338.0,35437,3.1,Subj 1 then Subj 2,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[[[59.557187688302, 21.40206507656825], [56.81...","[[56.44453190593808, 22.041576176751878], [54....","[13.304398, 13.279373, 13.2186365, 13.117102, ...","[[62.308924226592204, 17.97009425079981], [62....","[12.874714, 12.780252, 12.830084, 12.994184, 1...","[27.327261, 28.553148, 29.562248, 30.353464, 3...","[26.742891, 27.9868, 29.002823, 29.783545, 30....","[27.911633, 29.1195, 30.121674, 30.923384, 31....","[200, 201, 202, 203, 204, 205, 206, 207, 208, ...","[35277308, 35277308, 35278694, 35280080, 35280..."
46,36337,36417,36437.0,36536,3.3,Subj 2 Only,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[[[62.31907642599725, 33.81738396661642], [60....","[[60.32622249671731, 35.42778006327852], [60.3...","[0.518957, 0.6057527, 0.6832184, 0.77924526, 0...","[[66.78737877919696, 32.823619089907766], [66....","[26.3084, 26.333275, 26.375715, 26.454876, 26....","[1.0364465, 1.3400655, 1.656862, 1.9446498, 2....","[0.672737, 0.96411556, 1.2816203, 1.5727413, 1...","[1.6744555, 1.9707181, 2.2606552, 2.5124037, 2...","[200, 201, 202, 203, 204, 205, 206, 207, 208, ...","[36377688, 36379074, 36380460, 36380460, 36381..."


In [80]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"]["trial_frame_index"].iloc[0].shape

(200,)

In [81]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=sleap_columns + ["cluster_timestamp"], errors="ignore")

In [82]:
for col in TRIALS_AND_SPECTRAL_DF.columns:
    print(col)

tone_start_frame
reward_start
reward_dispensed
tone_stop_frame
condition
competition_closeness
get_reward_frame
out_reward_frame
experiment
video_name
tone_start_timestamp
tone_stop_timestamp
cohort
session_dir
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
notes
session_path
recording
current_subject
subject
all_subjects
first_timestamp
last_timestamp
video_timestamps
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
vHPC_lfp_trace
BLA_lfp_trace
LH_lfp_trace
MD_lfp_trace
mPFC_lfp_trace
vHPC_theta_band
BLA_theta_band
LH_theta_band
MD_theta_band
mPFC_theta_band
vHPC_gamma_band
BLA_gamma_band
LH_gamma_band
MD_gamma_band
mPFC_gamma_band
vHPC_theta_phase
BLA_theta_phase
LH_theta_phase
MD_theta_phase
mPFC_theta_phase
vHPC_gamma_phase
BLA_gamma_phase
LH_gamma_phase
MD_gamma_phase
mPFC_gamma_phase
vHPC_power_all_frequencies_all_windows
BLA_power_all_frequencies_all_windows
LH_power_all_frequencies_all_windows
MD_power_all_frequencies_all_windo

- Filtering coherence

In [83]:
coherence_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "coherence" in col and "timestamps" not in col and "calculation" not in col]

In [84]:
coherence_columns

['BLA_LH_coherence_all_frequencies_all_windows',
 'BLA_MD_coherence_all_frequencies_all_windows',
 'BLA_mPFC_coherence_all_frequencies_all_windows',
 'BLA_vHPC_coherence_all_frequencies_all_windows',
 'LH_MD_coherence_all_frequencies_all_windows',
 'LH_mPFC_coherence_all_frequencies_all_windows',
 'LH_vHPC_coherence_all_frequencies_all_windows',
 'MD_mPFC_coherence_all_frequencies_all_windows',
 'MD_vHPC_coherence_all_frequencies_all_windows',
 'mPFC_vHPC_coherence_all_frequencies_all_windows',
 'BLA_LH_coherence_theta',
 'BLA_LH_coherence_gamma',
 'BLA_MD_coherence_theta',
 'BLA_MD_coherence_gamma',
 'BLA_mPFC_coherence_theta',
 'BLA_mPFC_coherence_gamma',
 'BLA_vHPC_coherence_theta',
 'BLA_vHPC_coherence_gamma',
 'LH_MD_coherence_theta',
 'LH_MD_coherence_gamma',
 'LH_mPFC_coherence_theta',
 'LH_mPFC_coherence_gamma',
 'LH_vHPC_coherence_theta',
 'LH_vHPC_coherence_gamma',
 'MD_mPFC_coherence_theta',
 'MD_mPFC_coherence_gamma',
 'MD_vHPC_coherence_theta',
 'MD_vHPC_coherence_gamma',


In [85]:
for col in sorted(coherence_columns):
    updated_item_col = "trial_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_coherence_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)

trial_BLA_LH_coherence_all_frequencies_all_windows
trial_BLA_LH_coherence_gamma
trial_BLA_LH_coherence_theta
trial_BLA_MD_coherence_all_frequencies_all_windows
trial_BLA_MD_coherence_gamma
trial_BLA_MD_coherence_theta
trial_BLA_mPFC_coherence_all_frequencies_all_windows
trial_BLA_mPFC_coherence_gamma
trial_BLA_mPFC_coherence_theta
trial_BLA_vHPC_coherence_all_frequencies_all_windows
trial_BLA_vHPC_coherence_gamma
trial_BLA_vHPC_coherence_theta
trial_LH_MD_coherence_all_frequencies_all_windows
trial_LH_MD_coherence_gamma
trial_LH_MD_coherence_theta
trial_LH_mPFC_coherence_all_frequencies_all_windows
trial_LH_mPFC_coherence_gamma
trial_LH_mPFC_coherence_theta
trial_LH_vHPC_coherence_all_frequencies_all_windows
trial_LH_vHPC_coherence_gamma
trial_LH_vHPC_coherence_theta
trial_MD_mPFC_coherence_all_frequencies_all_windows
trial_MD_mPFC_coherence_gamma
trial_MD_mPFC_coherence_theta
trial_MD_vHPC_coherence_all_frequencies_all_windows
trial_MD_vHPC_coherence_gamma
trial_MD_vHPC_coherence_thet

In [86]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=coherence_columns + ["coherence_timestamps"], errors="ignore")

- Filtering Grangers

In [87]:
granger_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "granger" in col and "timestamps" not in col and "calculation" not in col]

In [88]:
granger_columns

['BLA_LH_granger_all_frequencies_all_windows',
 'LH_BLA_granger_all_frequencies_all_windows',
 'BLA_MD_granger_all_frequencies_all_windows',
 'MD_BLA_granger_all_frequencies_all_windows',
 'BLA_mPFC_granger_all_frequencies_all_windows',
 'mPFC_BLA_granger_all_frequencies_all_windows',
 'BLA_vHPC_granger_all_frequencies_all_windows',
 'vHPC_BLA_granger_all_frequencies_all_windows',
 'LH_MD_granger_all_frequencies_all_windows',
 'MD_LH_granger_all_frequencies_all_windows',
 'LH_mPFC_granger_all_frequencies_all_windows',
 'mPFC_LH_granger_all_frequencies_all_windows',
 'LH_vHPC_granger_all_frequencies_all_windows',
 'vHPC_LH_granger_all_frequencies_all_windows',
 'MD_mPFC_granger_all_frequencies_all_windows',
 'mPFC_MD_granger_all_frequencies_all_windows',
 'MD_vHPC_granger_all_frequencies_all_windows',
 'vHPC_MD_granger_all_frequencies_all_windows',
 'mPFC_vHPC_granger_all_frequencies_all_windows',
 'vHPC_mPFC_granger_all_frequencies_all_windows',
 'BLA-to-LH_granger_theta',
 'BLA-to-LH_

In [89]:
for col in sorted(granger_columns):
    updated_item_col = "trial_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_granger_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)

trial_BLA-to-LH_granger_gamma
trial_BLA-to-LH_granger_theta
trial_BLA-to-MD_granger_gamma
trial_BLA-to-MD_granger_theta
trial_BLA-to-mPFC_granger_gamma
trial_BLA-to-mPFC_granger_theta
trial_BLA-to-vHPC_granger_gamma
trial_BLA-to-vHPC_granger_theta
trial_BLA_LH_granger_all_frequencies_all_windows
trial_BLA_MD_granger_all_frequencies_all_windows
trial_BLA_mPFC_granger_all_frequencies_all_windows
trial_BLA_vHPC_granger_all_frequencies_all_windows
trial_LH-to-BLA_granger_gamma
trial_LH-to-BLA_granger_theta
trial_LH-to-MD_granger_gamma
trial_LH-to-MD_granger_theta
trial_LH-to-mPFC_granger_gamma
trial_LH-to-mPFC_granger_theta
trial_LH-to-vHPC_granger_gamma
trial_LH-to-vHPC_granger_theta
trial_LH_BLA_granger_all_frequencies_all_windows
trial_LH_MD_granger_all_frequencies_all_windows
trial_LH_mPFC_granger_all_frequencies_all_windows
trial_LH_vHPC_granger_all_frequencies_all_windows
trial_MD-to-BLA_granger_gamma
trial_MD-to-BLA_granger_theta
trial_MD-to-LH_granger_gamma
trial_MD-to-LH_granger_t

In [90]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=granger_columns + ["granger_timestamps"], errors="ignore")

In [91]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

- Filtering power

In [92]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,trial_vHPC-to-LH_granger_theta,trial_vHPC-to-MD_granger_gamma,trial_vHPC-to-MD_granger_theta,trial_vHPC-to-mPFC_granger_gamma,trial_vHPC-to-mPFC_granger_theta,trial_vHPC_BLA_granger_all_frequencies_all_windows,trial_vHPC_LH_granger_all_frequencies_all_windows,trial_vHPC_MD_granger_all_frequencies_all_windows,trial_vHPC_mPFC_granger_all_frequencies_all_windows,trial_granger_timestamps
0,1272,1352,1372.0,1471,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[nan, nan, nan, 0.43979117, nan, nan, nan, 0.7...","[0.42408258, nan, nan, 0.43337917, nan, nan, n...","[0.9244214, nan, nan, 0.75117475, nan, nan, na...","[0.5580725, 0.6544174, 0.44992533, 0.437711, 0...","[0.4442911, 0.42809182, 0.26317245, 0.31479073...","[[0.033909287, 0.49683708, 0.053923283, 0.6585...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[1.1425717, 4.1493745, 0.1925774, 0.35515466,...","[[0.9095126, 1.1860693, 0.22170861, 0.32518223...","[1280000, 1290000, 1300000, 1310000, 1320000, ..."
1,1272,1352,1372.0,1471,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[0.32359514, 0.36578676, 0.43302304, 0.4178053...","[0.3016257, 0.5170068, 0.5291108, 0.43605345, ...","[0.50897765, 0.5158619, 0.65584356, 0.19664682...","[0.13350372, 0.23661414, 0.29183108, 0.3712481...","[0.16222906, 0.2717419, 1.0775431, 0.2079854, ...","[[1.0026236, 0.51690346, 0.18239346, 0.0252298...","[[1.5948796, 0.46469027, 0.15642908, 0.0738552...","[[0.0375482, 0.050444312, 0.24785438, 0.215615...","[[0.0032223964, 0.021228366, 0.3698781, 0.3518...","[1280000, 1290000, 1300000, 1310000, 1320000, ..."
2,3268,3348,3368.0,3469,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[0.302084, 0.051128585, 0.21332674, 0.415743, ...","[0.4706317, 0.37053043, 0.29853183, 0.53845054...","[0.32494396, 0.109982185, 0.14266354, 1.227110...","[0.56407404, 0.26936522, 0.6307696, 0.3151489,...","[0.2484267, 0.6275155, 0.8340126, 0.09485431, ...","[[2.2062507, 0.2695673, 0.4097873, 0.02609093,...","[[0.3434687, 0.76559144, 0.5626536, 0.8007811,...","[[nan, 0.8665458, 0.9720444, 0.41512448, 0.321...","[[0.36356246, 0.122665815, 0.10617317, 0.30965...","[3280000, 3290000, 3300000, 3310000, 3320000, ..."
3,3268,3348,3368.0,3469,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[nan, nan, 0.4760641, 0.30736396, 0.18894514, ...","[nan, nan, 0.2416683, 0.6191683, 0.8658184, 0....","[nan, nan, 0.7860084, 0.6086193, 0.17916876, 0...","[nan, nan, 0.14414899, 0.28585237, 0.15822333,...","[nan, nan, 0.6812989, 0.27047354, 0.2221948, 0...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[3280000, 3290000, 3300000, 3310000, 3320000, ..."
4,4466,4546,4566.0,4667,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[0.27652302, nan, nan, 0.25373647, 0.3350416, ...","[0.2581781, 0.38288853, 0.33808008, 0.25682807...","[0.32289532, 0.3066461, 0.44500187, 0.4600196,...","[0.26871485, 0.45134282, 0.59286803, 0.3960509...","[0.19578956, 0.29618084, 0.4881647, 0.42467085...","[[0.7267522, 0.49278006, 0.074722454, 0.145846...","[[1.0748537, 0.15602921, 0.05029094, 0.1333028...","[[1.986834, 0.66492647, 0.40491146, 0.07051679...","[[3.85709, 0.24011524, 0.29333848, 0.21249528,...","[4480000, 4490000, 4500000, 4510000, 4520000, ..."


In [93]:
power_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "power" in col and "timestamps" not in col and "calculation" not in col]

In [94]:
power_columns

['vHPC_power_all_frequencies_all_windows',
 'BLA_power_all_frequencies_all_windows',
 'LH_power_all_frequencies_all_windows',
 'MD_power_all_frequencies_all_windows',
 'mPFC_power_all_frequencies_all_windows',
 'vHPC_power_theta',
 'vHPC_power_gamma',
 'BLA_power_theta',
 'BLA_power_gamma',
 'LH_power_theta',
 'LH_power_gamma',
 'MD_power_theta',
 'MD_power_gamma',
 'mPFC_power_theta',
 'mPFC_power_gamma']

In [95]:
for col in sorted(power_columns):
    updated_item_col = "trial_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_power_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)


trial_BLA_power_all_frequencies_all_windows
trial_BLA_power_gamma
trial_BLA_power_theta
trial_LH_power_all_frequencies_all_windows
trial_LH_power_gamma
trial_LH_power_theta
trial_MD_power_all_frequencies_all_windows
trial_MD_power_gamma
trial_MD_power_theta
trial_mPFC_power_all_frequencies_all_windows
trial_mPFC_power_gamma
trial_mPFC_power_theta
trial_vHPC_power_all_frequencies_all_windows
trial_vHPC_power_gamma
trial_vHPC_power_theta


In [96]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=power_columns + ["power_timestamps"], errors="ignore")

In [97]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

tone_start_frame
reward_start
reward_dispensed
tone_stop_frame
condition
competition_closeness
get_reward_frame
out_reward_frame
experiment
video_name
tone_start_timestamp
tone_stop_timestamp
cohort
session_dir
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
notes
session_path
recording
current_subject
subject
all_subjects
first_timestamp
last_timestamp
video_timestamps
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
vHPC_lfp_trace
BLA_lfp_trace
LH_lfp_trace
MD_lfp_trace
mPFC_lfp_trace
vHPC_theta_band
BLA_theta_band
LH_theta_band
MD_theta_band
mPFC_theta_band
vHPC_gamma_band
BLA_gamma_band
LH_gamma_band
MD_gamma_band
mPFC_gamma_band
vHPC_theta_phase
BLA_theta_phase
LH_theta_phase
MD_theta_phase
mPFC_theta_phase
vHPC_gamma_phase
BLA_gamma_phase
LH_gamma_phase
MD_gamma_phase
mPFC_gamma_phase
power_calculation_frequencies
coherence_calculation_frequencies
granger_calculation_frequencies
tone_start_frame_remove
agent_tail_base
agent_to_rew

# Filtering out phase

In [98]:
lfp_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "trace" in col and "timestamps" not in col and "calculation" not in col]

In [99]:
band_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "band" in col and "timestamps" not in col and "calculation" not in col]

In [100]:
phase_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "phase" in col and "timestamps" not in col and "calculation" not in col]

In [101]:
phase_columns = phase_columns + band_columns +lfp_columns

In [102]:
phase_columns

['vHPC_theta_phase',
 'BLA_theta_phase',
 'LH_theta_phase',
 'MD_theta_phase',
 'mPFC_theta_phase',
 'vHPC_gamma_phase',
 'BLA_gamma_phase',
 'LH_gamma_phase',
 'MD_gamma_phase',
 'mPFC_gamma_phase',
 'vHPC_theta_band',
 'BLA_theta_band',
 'LH_theta_band',
 'MD_theta_band',
 'mPFC_theta_band',
 'vHPC_gamma_band',
 'BLA_gamma_band',
 'LH_gamma_band',
 'MD_gamma_band',
 'mPFC_gamma_band',
 'vHPC_lfp_trace',
 'BLA_lfp_trace',
 'LH_lfp_trace',
 'MD_lfp_trace',
 'mPFC_lfp_trace']

In [103]:
for col in sorted(phase_columns):
    updated_item_col = "trial_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_lfp_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[0], axis=1)


trial_BLA_gamma_band
trial_BLA_gamma_phase
trial_BLA_lfp_trace
trial_BLA_theta_band
trial_BLA_theta_phase
trial_LH_gamma_band
trial_LH_gamma_phase
trial_LH_lfp_trace
trial_LH_theta_band
trial_LH_theta_phase
trial_MD_gamma_band
trial_MD_gamma_phase
trial_MD_lfp_trace
trial_MD_theta_band
trial_MD_theta_phase
trial_mPFC_gamma_band
trial_mPFC_gamma_phase
trial_mPFC_lfp_trace
trial_mPFC_theta_band
trial_mPFC_theta_phase
trial_vHPC_gamma_band
trial_vHPC_gamma_phase
trial_vHPC_lfp_trace
trial_vHPC_theta_band
trial_vHPC_theta_phase


In [104]:
TRIALS_AND_SPECTRAL_DF["trial_lfp_timestamps"]

0      [1275900, 1275920, 1275940, 1275960, 1275980, ...
1      [1275900, 1275920, 1275940, 1275960, 1275980, ...
2      [3275920, 3275940, 3275960, 3275980, 3276000, ...
3      [3275920, 3275940, 3275960, 3275980, 3276000, ...
4      [4475940, 4475960, 4475980, 4476000, 4476020, ...
                             ...                        
871    [25464500, 25464520, 25464540, 25464560, 25464...
872    [26564520, 26564540, 26564560, 26564580, 26564...
873    [26564520, 26564540, 26564560, 26564580, 26564...
874    [28264540, 28264560, 28264580, 28264600, 28264...
875    [28264540, 28264560, 28264580, 28264600, 28264...
Name: trial_lfp_timestamps, Length: 876, dtype: object

In [105]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=phase_columns + ["lfp_timestamps"], errors="ignore")

In [106]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [107]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

tone_start_frame
reward_start
reward_dispensed
tone_stop_frame
condition
competition_closeness
get_reward_frame
out_reward_frame
experiment
video_name
tone_start_timestamp
tone_stop_timestamp
cohort
session_dir
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
notes
session_path
recording
current_subject
subject
all_subjects
first_timestamp
last_timestamp
video_timestamps
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
power_calculation_frequencies
coherence_calculation_frequencies
granger_calculation_frequencies
tone_start_frame_remove
agent_tail_base
agent_to_reward_port_angle
subject_tail_base
subject_to_reward_port_angle
to_reward_port_angle_diff
to_reward_port_angle_sum
within_trial_index
agent
10s_after_tone_frame
10s_before_tone_frame
all_subjects_remove
body_parts
box_bottom_left
box_bottom_right
box_number
box_top_left
box_top_right
color
comp_id
competition_closeness_remove
condition 
experiment_remove
first_timestamp_remove
get_reward_frame_

In [110]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"]["trial_kmeans_cluster"].iloc[0].shape

(200,)

In [111]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [112]:
raise ValueError()

ValueError: 

# Filtering for rows that are in the video

In [ ]:
TRIALS_AND_SPECTRAL_DF["in_video"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: x["start_frame"] <= x["trial_video_frame"][0] <= x["stop_frame"], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["in_video"]].reset_index()

In [ ]:
TRIALS_AND_SPECTRAL_DF

In [ ]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
raise ValueError()

# Filtering out spikes

In [ ]:
raise ValueError()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["tone_start_timestamp"], stop=x["baseline_stop_timestamp"]).astype(int), axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"]).astype(int), axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_stop_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"].iloc[0]

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1], axis=1)
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1].T, axis=1)
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=["spike_clusters", "spike_times", "neuron_average_fr", "neuron_average_timestamps",], errors="ignore")

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"].iloc[0].shape

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"].iloc[0].shape